In [ ]:
 from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Dependencies

In [ ]:
! pip install "transformers==3.3.0"
! pip install gcsfs
! pip install pytrec_eval

     |████████████████████████████████| 1.1MB 6.4MB/s 
     |████████████████████████████████| 870kB 17.8MB/s 
     |████████████████████████████████| 1.2MB 14.4MB/s 
     |████████████████████████████████| 3.0MB 41.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=96f1805328aba6893f1eeacca64c750124d04bb6c1cca571e39fe056f21ba338
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 184kB 8.2MB/s 
     |████████████████████████████████| 1.3MB 15.4MB/s 
     |████████████████████████████████| 112kB 38.2MB/s 
     |████████████████████████████████| 143kB 21.9MB/s 
     |████████████████████████████████| 296kB 25.1MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=264407 sha256=bd162d7b94bbac76f127eaf98cb99532e8dbd0f324cdfcfffed598a896d7df6f
  Stored in directory: /root/.

**Get the code**

In [ ]:
import sys
%cd /content/
!rm -r ExactMatchMarking
!test -d ExactMatchMarking || git clone https://github.com/BOUALILILila/ExactMatchMarking.git
import sys
if not 'ExactMatchMarking' in sys.path:
  sys.path += ['ExactMatchMarking']

/content
rm: cannot remove 'ExactMatchMarking': No such file or directory
Cloning into 'ExactMatchMarking'...
remote: Enumerating objects: 506, done.
remote: Counting objects: 100% (506/506), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 506 (delta 356), reused 313 (delta 163), pack-reused 0
Receiving objects: 100% (506/506), 122.67 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (356/356), done.


**Google Cloud Storage set up**

In [ ]:
import tensorflow as tf
tf.__version__


'2.4.1'

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
import os
os.environ['COLAB_SKIP_TPU_AUTH'] = '1'
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_ID="YOUR GCS PROJECT ID #@param {type:"string"}
bucket_name="BUCKET NAME" #@param {type:"string"}
!gcloud config set project {project_ID} 

Updated property [core/project].


## Parameters

In [ ]:
## Tain params
# if precise tokens use augmented model 
MODEL_NAME_OR_PATH = "google/electra-base-discriminator" #@param {type:"string"}
CHECKPOINT_NAME='electra_ckpts' #@param {type:"string"}


OUTPUT_DIR = "gs://path/output/dir/to/save/tf_ckpts" #@param {type:"string"}
assert OUTPUT_DIR, 'Must specify an existing GCS bucket name'
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

COLLECTION='msmarco' #@param {type:"string"}

STRATEGY= "sim_pair" #@param {type:"string"}

  ## Train
TRAIN_DATA_DIR = "gs://path/to/train/dataset/tfrecord" #@param {type:"string"}
# Convention: {TRAIN_DATA_DIR}/dataset_train_{TRAIN_SET_NAME}.tf
TRAIN_SET_NAME= "msmarco_small_sim_pair" #@param {type:"string"}

# Must be on GCS
LOG_DIR = "gs://path/to/log/dir/on/GCS"#@param {type:"string"}

***** Model output directory: gs://lila_data/msmarco/electra_base *****


## Run the model

In [ ]:
import logging 
import os, glob, re
import pandas as pd
import tensorflow as tf
from transformers import (
    TF2_WEIGHTS_NAME,
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainingArguments,
)

from Modeling import (
    CustomTFTrainer,
    CustomTFTrainingArguments,
    get_eval_metric,
    wrap_relevance_head,
)
from args import (
    ModelArguments,
    DataTrainingArguments,
)

def get_logger(name, level=logging.INFO):
    handler = logging.StreamHandler(stream=sys.stderr)
    formatter = logging.Formatter(
        fmt="%(asctime)s: %(funcName)s +%(lineno)s: %(levelname)-8s %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)
    return logger 

logger = get_logger('Train')


training_args = CustomTFTrainingArguments(
    output_dir = OUTPUT_DIR,
    do_train = True,
    do_eval = False,
    do_predict = False,
    do_early_stopping = False,
    evaluate_during_training = False,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 4,
    learning_rate= 3e-6,
    weight_decay = 0.01,
    adam_epsilon = 1e-6,
    num_train_epochs = -1,
    max_steps = 100000,
    warmup_steps = 10000,
    logging_steps = 1000,
    save_steps = 5000,
    tpu_name = f"grpc://{os.environ['COLAB_TPU_ADDR']}",
    tf_ckpt_dir = f'{OUTPUT_DIR}/{CHECKPOINT_NAME}/{STRATEGY}', # where to save the TF2 pure trainable checkpoints
    ckpt_dir = f'/path/to/local/or/drive/{STRATEGY}_wrap', # where to save the Transformers library ckpt format easy to reuse 
    eval_all_checkpoints = False,
    logging_dir = LOG_DIR,
)


data_args = DataTrainingArguments(
    collection = COLLECTION,
    how = 'words',
    train_data_dir = TRAIN_DATA_DIR,
    # eval_data_dir = EVAL_DATA_DIR,
    train_set_name = TRAIN_SET_NAME,
    # eval_set_name = EVAL_SET_NAME,
    # test_set_name = TEST_SET_NAME,
    # out_suffix = OUT_SUFFIX,
    # eval_qrels_file = EVAL_QRELS_FILE_NAME,
    # test_qrels_file = TEST_QRELS_FILE_NAME,
    max_seq_length = 512,
)

model_args = ModelArguments(
    model_name_or_path = MODEL_NAME_OR_PATH,
)


def main():
    
    logger.info(
        "n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.n_gpu,
        bool(training_args.n_gpu > 1),
        training_args.fp16,
    )
    logger.info("Training/evaluation/prediction parameters %s", training_args)

    if training_args.do_train or training_args.do_eval:
       
        num_labels = 2
        output_mode = 'classification'

        # Load pretrained model and tokenizer
        #
        # Distributed training:
        # The .from_pretrained methods guarantee that only one local process can concurrently
        # download model & vocab.

        config = AutoConfig.from_pretrained(
            model_args.config_name if model_args.config_name else model_args.model_name_or_path,
            num_labels=num_labels,
            cache_dir=model_args.cache_dir,
            output_hidden_states=False,
        )

        with training_args.strategy.scope():
            orig_model = TFAutoModelForSequenceClassification.from_pretrained(
                model_args.model_name_or_path,
                from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
                config=config,
                cache_dir=model_args.cache_dir,
            )

            orig_classifier = orig_model.classifier
            # For ELECTRA need to change the classifier 
            model = wrap_relevance_head(orig_model)
            print('Verfiy wrapping')
            print(model.classifier)
            print(type(model))


        # Get datasets
        if training_args.do_train:
            filename = os.path.join(data_args.train_data_dir, f'dataset_train_{data_args.train_set_name}.tf')
            print('== filename train : ', filename)
            if tf.io.gfile.exists(filename):
                train_dataset, num_train_examples = data_args.doc_processor.get_train_dataset(filename, 
                                                                            training_args.train_batch_size, training_args.seed)
                print(num_train_examples)
            else:
                raise IOError('File does not exist: ', filename)
        else:
            train_dataset, num_train_examples = None, 0

        if training_args.do_eval or training_args.do_early_stopping:
            filename = os.path.join(data_args.eval_data_dir, f'dataset_dev_{data_args.eval_set_name}.tf')
            ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_dev_{data_args.eval_set_name}.tsv')
            if tf.io.gfile.exists(filename):
                eval_dataset, num_eval_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                                        training_args.eval_batch_size)
            else:
                raise IOError('File does not exist: ', filename)
            if tf.io.gfile.exists(ids_file):
                query_doc_ids = pd.read_csv(ids_file, 
                                            header=None, index_col=None, delimiter='\t', 
                                            names=['id','qid','did','pass'], 
                                            dtype={'id':str, 'qid':str,'did':str})
            else:
                raise IOError('File does not exist: ', ids_file)

            if data_args.eval_qrels_file is not None:
                qrels_file = os.path.join(data_args.eval_data_dir, f'{data_args.eval_qrels_file}.tsv')
                if tf.io.gfile.exists(qrels_file):
                    eval_qrels = pd.read_csv(qrels_file,
                                header=None, index_col=None, delimiter='\t', names=['qid','did','label'], 
                                dtype={'qid':str,'did':str, 'label':int})
                else:
                    raise IOError('File does not exist: ', qrels_file)
            else:
                eval_qrels = None
        else:
            eval_dataset, num_eval_examples, query_doc_ids, eval_qrels = None, 0, None, None

        # eval_metric = get_eval_metric(data_args.collection)


        # Initialize our Trainer
        trainer = CustomTFTrainer(
            model = model,
            args = training_args,
            train_dataset = train_dataset,
            num_train_examples = num_train_examples,
            eval_dataset = eval_dataset,
            num_eval_examples = num_eval_examples,
            out_suffix = f'{data_args.eval_set_name}_{data_args.out_suffix}', # eval output file name
            # eval_metric = eval_metric,
            # compute_metrics = eval_metric.compute_on_df,
            query_doc_ids = query_doc_ids,
            eval_qrels = eval_qrels,
        )

        # Training
        if training_args.do_train:
            trainer.train()

            if not training_args.do_early_stopping:
                trainer.save_model()
                # You can restore the original classifier architecture and change the trained weigths 
                # to keep a ckpt that can be loaded with the standard TFAutoModelForSequenceClassification (but need to call the wrap_relevance_head)
                # Or use the first ckpt saved above with trainer.save_model() that needs to be loaded with the custom TFElectraForRelevanceClassification class
                trained_model = trainer.model
                orig_classifier.dropout, orig_classifier.out_proj = trained_model.classifier.dropout,  trained_model.classifier.out_proj
                trained_model.classifier = orig_classifier
                trained_model.save_pretrained(f'/path/local/drive/{STRATEGY}_wrap_unwrap')
                

        # Evaluation        
        if training_args.do_eval:
            logger.info("*** Evaluate ***")

            sstep = 'final' if training_args.do_train else 0
            results = trainer.evaluate(step=sstep)
            print(sstep)
            print(results)

    # Test
    if training_args.do_predict:

        checkpoints = list(
                os.path.dirname(c)
                for c in sorted(
                    glob.glob(f'{training_args.ckpt_dir}' + "/**/" + TF2_WEIGHTS_NAME, recursive=True),
                    key=lambda f: int("".join(filter(str.isdigit, f)) or -1),
                )
            )

        if len(checkpoints) == 0:
            raise IOError('No checkpoint found at this location: ', training_args.ckpt_dir)

        elif not training_args.eval_all_checkpoints:
            if training_args.ckpt_dir in checkpoints:
                checkpoints = [training_args.ckpt_dir]
            else:
                checkpoints = [checkpoints[-1]]    
                    
        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        filename = os.path.join(data_args.eval_data_dir, f'dataset_test_{data_args.test_set_name}.tf')
        if tf.io.gfile.exists(filename):
            test_dataset, num_test_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                    training_args.eval_batch_size)
        else: 
            raise IOError('File does not exist: ', filename)

        ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_test_{data_args.test_set_name}.tsv')
        if tf.io.gfile.exists(ids_file):
            query_doc_ids = pd.read_csv(ids_file,
                                header=None, index_col=None, delimiter='\t', 
                                names=['id','qid','did','pass'],
                                dtype={'id':str, 'qid':str,'did':str})
        else: 
            raise IOError('File does not exist: ', ids_file)

        if data_args.test_qrels_file:
            qrels_file = os.path.join(data_args.eval_data_dir, f'{data_args.test_qrels_file}.tsv')
            if tf.io.gfile.exists(qrels_file):
                test_qrels = pd.read_csv(qrels_file, 
                            header=None, index_col=None, delimiter='\t', 
                            names=['qid','did','label'], 
                            dtype={'qid':str,'did':str, 'label':int})
                
            else :
                raise IOError('File does not exist: ', qrels_file)
        else:
            test_qrels = None

        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if re.match(".*checkpoint*-[0-9]", checkpoint) else "final"
            logger.info("Evaluate the following checkpoint-step: %s - %s", checkpoint, global_step)
            print("Evaluate the following checkpoint-step:", checkpoint, global_step)

            with training_args.strategy.scope():
                # if using the wrap_unwrap ckpt
                trained_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
                trained_model = wrap_relevance_head(trained_model)
                # # if using the wrap ckpt ==> modified classifier
                # trained_model = TFElectraForRelevanceClassification.from_pretrained(checkpoint)
        
            trainer = CustomTFTrainer(
                          model = trained_model,
                          args = training_args,
                      )

            trainer.predict(test_dataset, num_test_examples, query_doc_ids, test_qrels, 
                            f'{data_args.test_set_name}_{data_args.out_suffix}-{global_step}')

if __name__ == "__main__":
    main()

INFO:tensorflow:Initializing the TPU system: grpc://10.97.79.42:8470


ExactMatchMarking/Modeling/training_args.py:108: UserWarning: TrainingArguments: (gs://lila_data/msmarco/electra_base/electra_ckpts/pre_pair_wrap) already exists and is not empty.                 Continuing training from last tf checkpoint if exists.                 Use --overwrite_tf_ckpt_dir to overwrite existing checkpoints (restart training).
  Use --overwrite_tf_ckpt_dir to overwrite existing checkpoints (restart training)."
/usr/local/lib/python3.7/dist-packages/transformers/training_args_tf.py:216: FutureWarning: The n_gpu argument is deprecated and will be removed in a future version, use n_replicas instead.
  FutureWarning,
INFO:tensorflow:Initializing the TPU system: grpc://10.97.79.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
16:59:41: main +104: INFO     n_gpu: 8, distributed training: True, 16-bits training: False
INFO:Train:n_gpu: 8, distributed training: True, 16-bits training: False
16:59:41: main +106: INFO     Training/evaluation/prediction parameters CustomTFTrainingArguments(output_dir='gs://lila_data/msmarco/electra_base', overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, evaluation_strategy='no', prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=3e-06, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-06, max_grad_norm=1.0, num_train_epochs=-1, max_steps=100000, warmup_steps=10000, logging_dir='gs://lila_data/log', logging_first_step=False, logging_steps=1000, s

verfiy wrapping
<class 'transformers.modeling_tf_electra.TFElectraForSequenceClassification'>
== filename train :  gs://lila_data/msmarco/electra_base/dataset_train_msmarco_small_pre_pair.tf
79561622


INFO:absl:Checkpoint file gs://lila_data/msmarco/electra_base/electra_ckpts/pre_pair_wrap/ckpt-11 found and restoring from checkpoint
INFO:absl:  Continuing training from checkpoint, will skip to saved global_step
INFO:absl:  Continuing training from epoch 0
INFO:absl:  Continuing training from global step 55000
INFO:absl:  Will skip the first 55000 steps in the 1 epoch
INFO:absl:***** Running training *****
INFO:absl:  Num examples = 79561622
INFO:absl:  Num Epochs = 1
INFO:absl:  Total optimization steps = 100000
INFO:absl:Starting Epoch 1 ...
INFO:absl:Skipping the first 55000 trained steps ...
INFO:absl:55000 training steps skipped. Resuming training at step 55000...
INFO:absl:Epoch 1 Step 56000 Train Loss 0.2038
INFO:absl:Epoch 1 Step 57000 Train Loss 0.2035
INFO:absl:Epoch 1 Step 58000 Train Loss 0.1111
INFO:absl:Epoch 1 Step 59000 Train Loss 0.2006
INFO:absl:Epoch 1 Step 60000 Train Loss 0.1131
INFO:absl:Saving checkpoint for step 60000 at gs://lila_data/msmarco/electra_base/ele